In [1]:
# Ignore Warning
import warnings
warnings.filterwarnings('ignore')

In [2]:
# Import the necessary packages
import os, sys
import torch
import torchvision
import time
import cv2

# Add the module path to the sys.path
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

import torch.nn as nn 
import torch.nn.functional as F
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from PIL import Image
from torch.utils.data import Dataset, DataLoader

In [3]:
# Import local helper files
from utils.dataset import load

In [8]:
# Check if CUDA is available
if torch.cuda.is_available():
    # Get the number of GPUs
    num_gpus = torch.cuda.device_count()
    print(f"Number of available CUDA devices: {num_gpus}")
    
    # Print the name of each GPU
    for i in range(num_gpus):
        print(f"Device {i}: {torch.cuda.get_device_name(i)}")
else:
    print("CUDA is not available.")

# Set the device
DEVICE = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

Number of available CUDA devices: 1
Device 0: NVIDIA GeForce RTX 3050 Ti Laptop GPU


In [5]:
# Specify the shape of the inputs for our network
IMG_SIZE = 120
CROP_SIZE = 110
IMG_SHAPE = (IMG_SIZE, IMG_SIZE, 3)

In [6]:
# Specify the batch size, number of epochs, and the size of the chunks
BATCH_SIZE = 32
NUM_EPOCHS = 4
CHUNK_SIZE = 10000

# Predefined Functions

In [36]:
# Pair Dataset
class FurniturePairDataset(Dataset):
    def __init__(self, df, root_dir, transform=None):
        self.data_df = df
        self.root_dir = root_dir
        self.transform = transform
        self.images = []
        self.color_histograms = []
        self.edge_histograms = []
        self._load_images_and_features()

    def _load_images_and_features(self):
        for idx in range(len(self.data_df)):
            try: 
                img_name = os.path.join(self.root_dir, self.data_df.iloc[idx, 0])
                image = Image.open(img_name)
                self.images.append(image)
                self.color_histograms.append(self._extract_color_histogram(image))
                self.edge_histograms.append(self._extract_edge_histogram(image))
            except Exception as e:
                print(f"Error loading image {img_name}: {e}")

    def _extract_color_histogram(self, image):
        image = np.array(image.convert("RGB"))
        hist = cv2.calcHist([image], [0, 1, 2], None, [8, 8, 8], [0, 256, 0, 256, 0, 256])
        cv2.normalize(hist, hist)
        return hist.flatten()

    def _extract_edge_histogram(self, image):
        image = np.array(image.convert("L"))
        edges = cv2.Canny(image, 100, 200)
        hist = cv2.calcHist([edges], [0], None, [256], [0, 256])
        cv2.normalize(hist, hist)
        return hist.flatten()

    def __len__(self):
        return len(self.data_df) * 2  # Each image will appear in one positive and one negative pair

    def __getitem__(self, idx):
        anchor_idx = idx // 2
        anchor_image = self.images[anchor_idx]
        anchor_color_hist = self.color_histograms[anchor_idx]
        anchor_edge_hist = self.edge_histograms[anchor_idx]

        if self.transform:
            anchor_image = self.transform(anchor_image)

        if idx % 2 == 0:
            similar_idx = self._find_most_similar(anchor_idx, anchor_color_hist)
            similar_image = self.images[similar_idx]

            if self.transform:
                similar_image = self.transform(similar_image)

            return (anchor_image, similar_image), 1
        else:
            dissimilar_idx = self._find_most_dissimilar(anchor_idx, anchor_edge_hist)
            dissimilar_image = self.images[dissimilar_idx]

            if self.transform:
                dissimilar_image = self.transform(dissimilar_image)

            return (anchor_image, dissimilar_image), 0

    def _find_most_similar(self, anchor_idx, anchor_hist):
        max_similarity = -1
        similar_idx = -1
        for idx in range(len(self.data_df)):
            if idx == anchor_idx:
                continue
            hist = self.color_histograms[idx]
            similarity = cv2.compareHist(anchor_hist, hist, cv2.HISTCMP_CORREL)
            if similarity > max_similarity:
                max_similarity = similarity
                similar_idx = idx
        return similar_idx

    def _find_most_dissimilar(self, anchor_idx, anchor_hist):
        min_similarity = 1
        dissimilar_idx = -1
        for idx in range(len(self.data_df)):
            if idx == anchor_idx:
                continue
            hist = self.edge_histograms[idx]
            similarity = cv2.compareHist(anchor_hist, hist, cv2.HISTCMP_CORREL)
            if similarity < min_similarity:
                min_similarity = similarity
                dissimilar_idx = idx
        return dissimilar_idx
    
# Get the Output Dimension of VGG16    
def get_output_shape(model, image_dim, device):
    model = model.to(device)  # Ensure the model is on the same device as the dummy input
    with torch.no_grad():
        dummy_input = torch.randn(1, 3, *image_dim).to(device)
        output = model(dummy_input)
    return output.shape[1]

# Load Dataset

In [13]:
# Path to the dataset
dataPath = '../../Data/Processed_Data/'

In [11]:
# Load DataFrame
furniture_dataset = load(dataPath)

Error processing ../../Data/Furniture_Data/lamps\Modern\11286modern-lighting.jpg: [Errno 13] Permission denied: '../../Data/Furniture_Data/lamps\\Modern\\11286modern-lighting.jpg'


In [12]:
# Check the shape of the dataset
furniture_dataset

,ImgPath,FileType,Width,Height,Ratio,Mode,Bands,Transparency,Animated,Category,Interior_Style
0,beds\Asian\19726asian-daybeds.jpg,jpg,350,350,1.0,RGB,R G B,False,False,beds,Asian
1,beds\Asian\20027asian-canopy-beds.jpg,jpg,350,350,1.0,RGB,R G B,False,False,beds,Asian
2,beds\Asian\20109asian-panel-beds.jpg,jpg,350,350,1.0,RGB,R G B,False,False,beds,Asian
3,beds\Asian\20508asian-platform-beds.jpg,jpg,350,350,1.0,RGB,R G B,False,False,beds,Asian
4,beds\Asian\20750asian-comforters-and-comforter...,jpg,350,350,1.0,RGB,R G B,False,False,beds,Asian
...,...,...,...,...,...,...,...,...,...,...,...
90078,tables\Victorian\5victorian-side-tables-and-en...,jpg,350,350,1.0,RGB,R G B,False,False,tables,Victorian
90079,tables\Victorian\6victorian-side-tables-and-en...,jpg,350,350,1.0,RGB,R G B,False,False,tables,Victorian
90080,tables\Victorian\7victorian-side-tables-and-en...,jpg,350,350,1.0,RGB,R G B,False,False,tables,Victorian
90081,tables\Victorian\8victorian-dining-tables.jpg,jpg,350,350,1.0,RGB,R G B,False,False,tables,Victorian


In [ ]:
# Transformation logic
transform = torchvision.transforms.Compose([
    torchvision.transforms.Resize((120, 120)),
    torchvision.transforms.RandomCrop((110, 110)),
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

# Model Definition

In [18]:
# Define the VGG16 used for feature extraction
class VGG16FeatureExtractor(nn.Module):
    def __init__(self):
        super(VGG16FeatureExtractor, self).__init__()
        
        self.block_1 = nn.Sequential(
                nn.Conv2d(in_channels=3,
                          out_channels=64,
                          kernel_size=(3, 3),
                          stride=(1, 1),
                          padding=1),
                nn.ReLU(),
                nn.Conv2d(in_channels=64,
                          out_channels=64,
                          kernel_size=(3, 3),
                          stride=(1, 1),
                          padding=1),
                nn.ReLU(),
                nn.MaxPool2d(kernel_size=(2, 2),
                             stride=(2, 2))
        )
        
        self.block_2 = nn.Sequential(
                nn.Conv2d(in_channels=64,
                          out_channels=128,
                          kernel_size=(3, 3),
                          stride=(1, 1),
                          padding=1),
                nn.ReLU(),
                nn.Conv2d(in_channels=128,
                          out_channels=128,
                          kernel_size=(3, 3),
                          stride=(1, 1),
                          padding=1),
                nn.ReLU(),
                nn.MaxPool2d(kernel_size=(2, 2),
                             stride=(2, 2))
        )
        
        self.block_3 = nn.Sequential(        
                nn.Conv2d(in_channels=128,
                          out_channels=256,
                          kernel_size=(3, 3),
                          stride=(1, 1),
                          padding=1),
                nn.ReLU(),
                nn.Conv2d(in_channels=256,
                          out_channels=256,
                          kernel_size=(3, 3),
                          stride=(1, 1),
                          padding=1),
                nn.ReLU(),        
                nn.Conv2d(in_channels=256,
                          out_channels=256,
                          kernel_size=(3, 3),
                          stride=(1, 1),
                          padding=1),
                nn.ReLU(),
                nn.MaxPool2d(kernel_size=(2, 2),
                             stride=(2, 2))
        )
        
        self.block_4 = nn.Sequential(   
                nn.Conv2d(in_channels=256,
                          out_channels=512,
                          kernel_size=(3, 3),
                          stride=(1, 1),
                          padding=1),
                nn.ReLU(),        
                nn.Conv2d(in_channels=512,
                          out_channels=512,
                          kernel_size=(3, 3),
                          stride=(1, 1),
                          padding=1),
                nn.ReLU(),        
                nn.Conv2d(in_channels=512,
                          out_channels=512,
                          kernel_size=(3, 3),
                          stride=(1, 1),
                          padding=1),
                nn.ReLU(),            
                nn.MaxPool2d(kernel_size=(2, 2),
                             stride=(2, 2))
        )
        
        self.block_5 = nn.Sequential(
                nn.Conv2d(in_channels=512,
                          out_channels=512,
                          kernel_size=(3, 3),
                          stride=(1, 1),
                          padding=1),
                nn.ReLU(),            
                nn.Conv2d(in_channels=512,
                          out_channels=512,
                          kernel_size=(3, 3),
                          stride=(1, 1),
                          padding=1),
                nn.ReLU(),            
                nn.Conv2d(in_channels=512,
                          out_channels=512,
                          kernel_size=(3, 3),
                          stride=(1, 1),
                          padding=1),
                nn.ReLU(),    
                nn.MaxPool2d(kernel_size=(2, 2),
                             stride=(2, 2))             
        )
        
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
            
        for m in self.modules():
            if isinstance(m, nn.Conv2d) or isinstance(m, nn.Linear):
                nn.init.kaiming_uniform_(m.weight, mode='fan_in', nonlinearity='relu')
                if m.bias is not None:
                    m.bias.detach().zero_()
                    
    def forward(self, x):
        x = self.block_1(x)
        x = self.block_2(x)
        x = self.block_3(x)
        x = self.block_4(x)
        x = self.block_5(x)
        x = self.avgpool(x)
        x = x.view(x.size(0), -1) # flatten
        return x

In [37]:
# Define the Siamese Network
class SiameseNetwork(nn.Module):
    def __init__(self, embedding_dim=48):
        super(SiameseNetwork, self).__init__()
        self.feature_extractor = VGG16FeatureExtractor()
        out_features = get_output_shape(self.feature_extractor, (IMG_SIZE, IMG_SIZE), DEVICE)
        self.feature_extractor.to(DEVICE)  # Move to device after getting output shape
        self.fc = nn.Linear(out_features, embedding_dim)
        
    def forward(self, x):
        x = self.feature_extractor(x)
        x = self.fc(x)
        return x

In [38]:
# Euclidean distance layer
class EuclideanDistance(nn.Module):
    def forward(self, featsA, featsB):
        return F.pairwise_distance(featsA, featsB, keepdim=True)

# Define the complete Siamese Network model
class SiameseModel(nn.Module):
    def __init__(self, embedding_dim=48):
        super(SiameseModel, self).__init__()
        self.feature_extractor = SiameseNetwork(embedding_dim)
        self.euclidean_distance = EuclideanDistance()
        self.fc = nn.Linear(1, 1)

    def forward(self, inputA, inputB):
        featsA = self.feature_extractor(inputA)
        featsB = self.feature_extractor(inputB)
        distance = self.euclidean_distance(featsA, featsB)
        output = torch.sigmoid(self.fc(distance))
        return output

In [40]:
# Initialize model, loss function, and optimizer
model = SiameseModel().to(DEVICE)
criterion = nn.BCELoss()
optimizer = torch.optim.SGD(model.parameters(), momentum=0.9, lr=0.01)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer,
                                                       factor=0.1,
                                                       mode='max',
                                                       verbose=True)

In [ ]:
# Function to process a chunk for training
def process_chunk_train(chunk_df, epoch, chunk_idx, num_chunks):
    # Create the pair dataset for the chunk
    pair_dataset = FurniturePairDataset(df=chunk_df, root_dir=dataPath, transform=transform)

    # Create the pair dataloader for the chunk
    pair_dataloader = DataLoader(pair_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=0)

    # Training loop for the chunk
    model.train()
    start_time = time.time()
    running_loss = 0.0
    total_batches = len(pair_dataloader)
    print(f"Total samples in this chunk: {len(chunk_df)}")
    print(f"Total batches in this chunk: {total_batches}")
    for batch_idx, ((imageA, imageB), labels) in enumerate(pair_dataloader):
        imageA, imageB, labels = imageA.cuda(), imageB.cuda(), labels.float().cuda()
        optimizer.zero_grad()
        outputs = model(imageA, imageB)
        loss = criterion(outputs, labels.unsqueeze(1))
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        if (batch_idx + 1) % 50 == 0:
            print(f"Epoch {epoch+1}/{NUM_EPOCHS}, Chunk {chunk_idx + 1}/{num_chunks}, Batch {batch_idx + 1}/{total_batches}, Loss: {running_loss / (batch_idx + 1)}")
    end_time = time.time()
    elapsed_time = end_time - start_time
    print(f"Epoch {epoch+1}/{NUM_EPOCHS}, Chunk {chunk_idx + 1}/{num_chunks} completed in {elapsed_time:.2f} seconds")


In [ ]:
# Process the dataset in chunks for training
for epoch in range(NUM_EPOCHS):
    num_chunks = len(furniture_dataset) // CHUNK_SIZE + 1
    for chunk_idx in range(num_chunks):
        chunk_df = furniture_dataset.iloc[chunk_idx * CHUNK_SIZE:(chunk_idx + 1) * CHUNK_SIZE]
        print(f"Processing training chunk {chunk_idx + 1}/{num_chunks}")
        process_chunk_train(chunk_df, epoch, chunk_idx, num_chunks)
    scheduler.step() 

In [ ]:
# Function to process a chunk for testing
def process_chunk_test(chunk_df):
    # Create the pair dataset for the chunk
    pair_dataset = FurniturePairDataset(df=chunk_df, root_dir=dataPath, transform=transform)

    # Create the pair dataloader for the chunk
    pair_dataloader = DataLoader(pair_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=0)

    # Testing loop for the chunk
    model.eval()
    start_time = time.time()
    running_loss = 0.0
    total_batches = len(pair_dataloader)
    with torch.no_grad():
        for batch_idx, ((imageA, imageB), labels) in enumerate(pair_dataloader):
            imageA, imageB, labels = imageA.cuda(), imageB.cuda(), labels.float().cuda()
            outputs = model(imageA, imageB)
            loss = criterion(outputs, labels.unsqueeze(1))
            running_loss += loss.item()
    end_time = time.time()
    elapsed_time = end_time - start_time
    print(f"Chunk Test completed in {elapsed_time:.2f} seconds, Test Loss: {running_loss/total_batches}")

In [ ]:
# Process the dataset in chunks for testing
num_chunks = len(furniture_dataset) // CHUNK_SIZE + 1
for chunk_idx in range(num_chunks):
    chunk_df = furniture_dataset.iloc[chunk_idx * CHUNK_SIZE:(chunk_idx + 1) * CHUNK_SIZE]
    print(f"Processing test chunk {chunk_idx + 1}/{num_chunks}")
    process_chunk_test(chunk_df)